In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

#hacktop package
from hacktops.data import generate_top_dataset
from hacktops.data import get_well_relevant_windows

# pyts
import pyts
from pyts.image import MarkovTransitionField
from pyts.image import GramianAngularField

# Tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

/usr/users/bdmagr4/ayobi/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
# defining necessary functions
setting_seed = 432

def random_sample_n(X, k):
    samples = random.sample(X, k=k)
    return samples

def data_viz(input_x, input_y, status):
    random.seed(setting_seed)
    true_samples = [input_x[i] for i in range(len(input_x)) if input_y[i]==status]
    samples = random_sample_n(true_samples, 32)
    fig, axes = plt.subplots(2, 8, figsize=(16,4))
    for i, ax in enumerate(axes.flat):
        data = samples[i]
        ax.plot(data)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show

# function to plot Gramian Angular images 

def gaf_viz(input_x, input_y, status):
    random.seed(setting_seed)
    true_samples = [input_x[i] for i in range(len(input_x)) if input_y[i]==status]
    samples = random_sample_n(true_samples, 32)
    fig, axes = plt.subplots(2, 8, figsize=(16,4))
    for i, ax in enumerate(axes.flat):
        data = samples[i]
        ax.imshow(data)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show


#1. Function to plot model's validation loss and validation accuracy
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

# fit and evaluate a model
def create_model():
    model = Sequential()
    #add model layers
    model.add(Conv2D(64, kernel_size=3, activation='sigmoid', input_shape=data_shape))
    model.add(Dropout(0.2))
    model.add(Conv2D(32, kernel_size=3, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Flatten())
    model.add(Dense(5, activation='sigmoid'))

    #compile model using accuracy to measure model performance
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# defining confusion matrix plot
def plot_matrix(matrix):
    group_names = ["True Neg","False Pos","False Neg","True Pos"]
    group_counts = ["{0:0.0f}".format(value) for value in matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in matrix.flatten()/np.sum(matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    plot = sns.heatmap(matrix, annot=labels, fmt="", cmap='Blues')
    return plot



In [3]:
import tslearn
from tslearn.metrics import dtw as tslearn_dtw
import pandas as pd
import hacktops
import numpy as np
def instance_norm(sample: np.array):
    s = (sample - np.min(sample)) / (np.max(sample) - np.min(sample) + 1)
    return s
from pyspark import SparkContext
from dtaidistance.dtw import distance as dtai_dtw
from dtaidistance.dtw import distance_fast as dtai_dtw_fast
from hacktops.settings import WINDOW_LENGTH
from tslearn.metrics import dtw as tslearn_dtw
from pyts.metrics import dtw as pyts_dtw
import random
from plotly.offline import iplot
from tqdm import tqdm
import numpy as np
from hacktops.data import generate_top_dataset
import matplotlib.pyplot as plt

class TopFinder:
    """
    TopFinder: wrapper for window classifier
    
    Limitations:
    - Work on single one top and assume independence among tops
    - Find top by classifying windows extracted from well data and discard
      the correlation between windows
    - Does not utilize geographical info of wells

    Usage example:

        >>> model.fit(dataset)
        >>> model.evaluate_windows = a_func

        >>> top_finder = TopFinder(model, top_name)
        >>> top_finder.examine_dataset(df_tops)

        >>> predicted_depth = top_finder.find_top(df_well)

    """

    def __init__(self, fitted_window_classifier, top_name):
        if fitted_window_classifier.evaluate_windows is None:
            raise ValueError("fitted_window_classifier has to have function evaluate_windows")
        self.window_classifier = fitted_window_classifier
        self.work_on_top = top_name
        self.stats = {}

    def examine_dataset(self, df_tops:pd.DataFrame):
        self.stats['top_depth_max'] = df_tops[self.work_on_top].max()
        self.stats['top_depth_min'] = df_tops[self.work_on_top].min()

    def extract_window(self, df_well:pd.DataFrame, center_idx, window_length):
        left_limit = center_idx - window_length
        right_limit = center_idx + window_length
        window = df_well.loc[left_limit : right_limit, 'GR'].to_numpy()
        return window

    def get_candidate_windows(self, df_well:pd.DataFrame):
        '''
        extra prior knowledge may be used to narrow down the scope of candidates, 
        e.g. top distribution. 

        return list of windows. Each window includes the depth of its center & GR data.
        '''
        max_, min_ = self.stats['top_depth_max'], self.stats['top_depth_min']
        center_  = (max_ + min_) / 2
        depth_diff_ = max_ - min_
        dilated_max_ =  center_ + DILATION_RATIO * depth_diff_ / 2
        dilated_min_ =  center_ - DILATION_RATIO * depth_diff_ / 2

        windows = []
        for idx, row in df_well.iterrows():
            if row['DEPTH'] < dilated_max_ and row['DEPTH'] > dilated_min_:
                window_depth = row['DEPTH']
                window_data = self.extract_window(df_well, idx, WINDOW_LENGTH)
                if window_data.shape != (WINDOW_LENGTH * 2 + 1,):
                    # print(window_data.shape) 
                    # It happens when the window gets out of the scope of well depth
                    continue
                windows.append((window_depth, window_data))
        return windows

    def select_window(self, windows, scores: np.array):
        '''
        extra prior knowledge may be used here, e.g. top relationships
        '''
        index_max = np.argmax(scores, axis=0)
        return windows[index_max]

    def find_top(self, df_well):
        """
        Step:
            1. Extract all candidate windows from the well
            2. Evalute each candidate by window classifier
            3. Select the best candidate
            4. Return its associated depth
        """
        if self.window_classifier is None:
            raise Exception("window_classifier is not set")
        if df_well.shape[0] == 0:
            raise Exception("input well has no data")

        self.windows = self.get_candidate_windows(df_well)
        print(f'{len(self.windows)} candidate windows')
        windows_data = np.array([w[1] for w in self.windows])
        self.scores = self.window_classifier.evaluate_windows(windows_data)
        selected_window = self.select_window(self.windows, self.scores)
        self.top_depth = selected_window[0]

        return self.top_depth


class SimpleDTWWindowEvaluator:
    """
    Simple DTW Window Evaluator

    Fit: Caches a set of real top windows

    Evaluate: Scores = [AVG(1 / 1 + dtw(candidate, real)) for each candidate]
    """
    def __init__(self, metric=tslearn_dtw, norm=instance_norm):
        self.metric = metric
        self.norm = norm

    def fit(self, real_top_windows):
        real_top_windows = np.array([self.norm(w) for w in real_top_windows])
        self._real_top_windows = real_top_windows

    def evaluate_windows(self, candidate_windows) -> np.array:
        scores = []
        candidate_windows = [self.norm(w) for w in candidate_windows]
        for w in tqdm(candidate_windows):
            dists = np.array([self.metric(_w, w) for _w in self._real_top_windows])
            weights = 1 / (1 + dists)
            scores.append(weights.sum() / weights.shape[0])
        scores = np.array(scores)
        return scores

class SimpleDTWWindowEvaluator_Spark:
    """
    Spark Version of Simple DTW Window Evaluator
    """
    def __init__(self, sc:SparkContext, metric=tslearn_dtw, norm=instance_norm):
        self.metric = metric
        self.sc = sc
        self.norm = norm

    def fit(self, real_top_windows):
        real_top_windows = np.array([self.norm(w) for w in real_top_windows])
        self._real_top_windows_rdd = self.sc.parallelize(real_top_windows)

    def evaluate_windows(self, candidate_windows):
        candidate_windows = [self.norm(w) for w in candidate_windows]
        input_rdd = self.sc.parallelize([(i, candidate_windows[i]) for i in range(len(candidate_windows))])
        aTuple = (0,0)
        metric = self.metric
        scores = input_rdd.cartesian(self._real_top_windows_rdd)\
                .map(lambda t : (t[0][0], 1 / (1 + metric(t[0][1], t[1]))))\
                .aggregateByKey(aTuple, lambda a,b: (a[0] + b,    a[1] + 1),
                                        lambda a,b: (a[0] + b[0], a[1] + b[1]))\
                .mapValues(lambda v: v[0] / v[1])\
                .sortByKey().map(lambda t : t[1])\
                .collect()
        return scores

# class NewModel:
#     def ...
#
#     def evaluate_windows(self, candidate_windows):
#         ...
#         return scores

def get_true_windows(df_logs, df_tops, top_, keep_depth = False):
    dataset = generate_top_dataset(df_logs=df_logs, df_tops=df_tops, top=top_)
    all_well_names = df_logs['wellName'].unique()
    print(f'{len(dataset[0])} windows extracted from {len(all_well_names)} wells')

    X = np.array(dataset[0]).squeeze(axis=2)
    y = np.array(dataset[1])
    
    print('X:', X.shape)
    print('y:', y.shape)

    true_idx = [idx for idx in range(len(X)) if y[idx] == True]
    print(f'{len(true_idx)} true windows left')

    return X[true_idx]

def get_true_depth(wellname, top, df_tops):
    return df_tops.loc[wellname, top]

def visual_scores(depths, scores, max_score_depth=None, true_depth=None, well_name=None):
    data = []
    data.append(go.Scatter(x=depths,y=scores))
    title = "Evaluation Score w.r.t depth"
    if well_name:
        title += f' [well: {well_name}]'
    fig = go.Figure(data=data, layout={'title':title})
    if max_score_depth:
        fig.add_vline(x=max_score_depth, line_width=2, line_color="yellow", \
            annotation_text='Predicated', annotation_position='top left')
    if true_depth:
        fig.add_vline(x=true_depth, line_width=2, line_color="green", \
            annotation_text='True', annotation_position='top right')
    return fig

def get_true_windows(df_logs, df_tops, top_, keep_depth = False):
    dataset = generate_top_dataset(df_logs=df_logs, df_tops=df_tops, top=top_)
    all_well_names = df_logs['wellName'].unique()
    print(f'{len(dataset[0])} windows extracted from {len(all_well_names)} wells')

    X = np.array(dataset[0]).squeeze(axis=2)
    y = np.array(dataset[1])
    
    print('X:', X.shape)
    print('y:', y.shape)

    true_idx = [idx for idx in range(len(X)) if y[idx] == True]
    print(f'{len(true_idx)} true windows left')

    return X[true_idx]

def get_true_depth(wellname, top, df_tops):
    return df_tops.loc[wellname, top]


def visual_scores(depths, scores, max_score_depth=None, true_depth=None, well_name=None):
    data = []
    data.append(go.Scatter(x=depths,y=scores))
    title = "Evaluation Score w.r.t depth"
    if well_name:
        title += f' [well: {well_name}]'
    fig = go.Figure(data=data, layout={'title':title})
    if max_score_depth:
        fig.add_vline(x=max_score_depth, line_width=2, line_color="yellow", \
            annotation_text='Predicated', annotation_position='top left')
    if true_depth:
        fig.add_vline(x=true_depth, line_width=2, line_color="green", \
            annotation_text='True', annotation_position='top right')
    return fig

In [4]:
top_ = 'MARCEL'
df_logs_ = pd.read_parquet("../data/logs.parquet")
df_loc_ = pd.read_parquet("../data/loc.parquet")
df_tops_ = pd.read_parquet("../data/tops.parquet")

df_logs_test_ = pd.read_parquet("../testdata/logs_50.parquet")
df_loc_test_ = pd.read_parquet("../testdata/loc_50.parquet")
df_tops_test_ = pd.read_csv("../testdata/tops_50.csv", index_col=0)

In [5]:
train_dataset = generate_top_dataset(df_logs=df_logs_, df_tops=df_tops_, top=top_)
test_dataset = generate_top_dataset(df_logs=df_logs_test_, df_tops=df_tops_test_, top=top_)

NAN FOUND


In [6]:
len(train_dataset[0][0])

61

In [8]:
X_train = np.array(train_dataset[0]).reshape(len(train_dataset[0]), len(test_dataset[0][0]))
y_train = np.array(train_dataset[1])
X_test = np.array(test_dataset[0]).reshape(len(test_dataset[0]), len(test_dataset[0][0]))
y_test = np.array(test_dataset[1])

print("Training set: ", X_train.shape,y_train.shape)
print("Testing set: ", X_test.shape,y_test.shape)

Training set:  (119800, 61) (119800,)
Testing set:  (10000, 61) (10000,)


In [ ]:
image_size_gaf = 40
gasf = GramianAngularField(image_size=image_size_gaf, method= 'summation')
gadf = GramianAngularField(image_size=image_size_gaf, method= 'difference')
mtf = MarkovTransitionField(image_size=image_size_gaf)

In [ ]:
# train dataset: converting train dataset to images
train_gasf_images = gasf.fit_transform(X_train)
train_gadf_images = gadf.fit_transform(X_train)
train_mtf_images = mtf.transform(X_train)

# test dataset: converting test dataset to images
test_gasf_images = gasf.fit_transform(X_test)
test_gadf_images = gadf.fit_transform(X_test)
test_mtf_images = mtf.transform(X_test)

In [ ]:
train_gasf_images.shape, train_gadf_images.shape, train_mtf_images.shape, test_gasf_images.shape, test_gadf_images.shape, test_mtf_images.shape

In [ ]:
# visualizing training data
data_viz(X_train, y_train, True)

In [ ]:
# visualizing test data
data_viz(X_test, y_test, True)

In [ ]:
data_viz(X_test, y_test, False)

In [ ]:
# training data
gaf_viz(train_gasf_images, y_train, True)

In [ ]:
# test data
gaf_viz(test_gasf_images, y_test, True)

In [ ]:
gaf_viz(train_gadf_images, y_train, True)

In [ ]:
gaf_viz(test_gadf_images, y_test, True)

In [ ]:
gaf_viz(train_mtf_images, y_train, True)

In [ ]:
gaf_viz(test_mtf_images, y_test, True)

In [ ]:
train_gasf_data = np.expand_dims(train_gasf_images, axis=3)
train_gadf_data = np.expand_dims(train_gadf_images, axis=3)
train_mtf_data = np.expand_dims(train_mtf_images, axis=3)
train_combined = np.concatenate((np.expand_dims(train_gasf_images, axis=3),np.expand_dims(train_gadf_images, axis=3), np.expand_dims(train_mtf_images, axis=3)), axis=3)

test_gasf_data = np.expand_dims(test_gasf_images, axis=3)
test_gadf_data = np.expand_dims(test_gadf_images, axis=3)
test_mtf_data = np.expand_dims(test_mtf_images, axis=3)
test_combined = np.concatenate((np.expand_dims(test_gasf_images, axis=3),np.expand_dims(test_gadf_images, axis=3), np.expand_dims(test_mtf_images, axis=3)), axis=3)

# GASF

In [ ]:
# choose the data
x_input = train_gasf_data

# selecting model shape
data_shape = x_input.shape[1:]
model = create_model()

In [ ]:
x_input.shape

In [ ]:

model_history = model.fit(x_input, y_train, epochs=15, batch_size=32, validation_split=0.30)

In [ ]:
model.load_weights(f"{top_}_gasf_model.h5")
#plot_model_history(model_history)

In [ ]:
# Evaluating the test data
model.evaluate(test_gasf_data, y_test)
y_predict = model.predict(test_gasf_data)
y_predict=np.argmax(y_predict,axis=1)

cf_matrix = confusion_matrix(y_predict,y_test)

print(classification_report(y_predict,y_test))


In [ ]:
def evaluate_windows(self, candidate_windows):
    candidate_windows = gasf.fit_transform(candidate_windows)
    candidate_windows.reshape(len(candidate_windows), 1)
    return self.evaluate(candidate_windows)
import types
model.evaluate_windows = types.MethodType(evaluate_windows, model)

In [ ]:
top_finder = TopFinder(model, top_)
top_finder.examine_dataset(df_tops_)

In [ ]:
test_well_names = df_logs_test_['wellName'].unique()
print(len(test_well_names))

In [ ]:
result = []
for test_well_name in tqdm(test_well_names):
    print(f'well: {test_well_name}')
    df_test_well = df_logs_test_[df_logs_test_['wellName'] == test_well_name]
    predicted_depth = top_finder.find_top(df_test_well)
    true_depth = get_true_depth(test_well_name, top_, df_tops_test_)
    result.append([test_well_name, predicted_depth, true_depth])
    # print(f'true depth: {true_depth}\t predicated depth: {predicted_depth}\t error: {abs(predicted_depth - true_depth)}')

In [ ]:
df_result = pd.DataFrame(result, columns=['wellName', 'predicated_depth', 'true_depth']).set_index('wellName')
df_tops_pred = df_result[['predicated_depth']].rename(columns={'predicated_depth': top_})
df_tops_true = df_result[['true_depth']].rename(columns={'true_depth': top_})

In [ ]:
from hacktops.evaluate import recall_tops

recall, mae, df_res = recall_tops(df_tops_true, df_tops_pred, tolerance = 4)
print("recall {0}, mae {1}".format(recall,mae))
df_res.head(50)

In [ ]:
depth = [w[0] for w in top_finder.windows]
fig = visual_scores(depth, top_finder.scores, top_finder.top_depth, true_depth, test_well_name)
iplot(fig)

# GADF

In [ ]:
# choose the data
x_input = x_gadf_data

# split in to test and train
x_train, x_test, y_train, y_test = train_test_split(x_input, Y, test_size=0.30, random_state=setting_seed)
# selecting model shape
data_shape = x_input.shape[1:]
model = create_model()
model_history = model.fit(x_train, y_train, epochs=15, batch_size=32, validation_split=0.30)


In [ ]:
model.save_weights(f"{top}_gadf_model.h5")
plot_model_history(model_history)

In [ ]:
# Evaluating the test data
model.evaluate(x_test, y_test)
y_predict = model.predict(x_test)
y_predict=np.argmax(y_predict,axis=1)

# confusion matrix
cf_matrix = confusion_matrix(y_predict,y_test)

# classification report
print(classification_report(y_predict,y_test))

# MTF

In [ ]:
# choose the data
x_input = x_mtf_data

# split in to test and train
x_train, x_test, y_train, y_test = train_test_split(x_input, Y, test_size=0.30, random_state=setting_seed)
# selecting model shape
data_shape = x_input.shape[1:]
model = create_model()
model_history = model.fit(x_train, y_train, epochs=15, batch_size=32, validation_split=0.30)

In [ ]:
model.save_weights(f"{top}_mtf_model.h5")
plot_model_history(model_history)

In [ ]:
# Evaluating the test data
model.evaluate(x_test, y_test)
y_predict = model.predict(x_test)
y_predict=np.argmax(y_predict,axis=1)

# confusion matrix
cf_matrix = confusion_matrix(y_predict,y_test)

# classification report
print(classification_report(y_predict,y_test))

# Combined 

In [ ]:
# choose the data
x_input = x_combined

# split in to test and train
x_train, x_test, y_train, y_test = train_test_split(x_input, Y, test_size=0.30, random_state=setting_seed)
# selecting model shape
data_shape = x_input.shape[1:]
model = create_model()
model_history = model.fit(x_train, y_train, epochs=15, batch_size=32, validation_split=0.30)

In [ ]:
model.save_weights(f"{top}_combined_model.h5")
plot_model_history(model_history)

In [ ]:
# Evaluating the test data
model.evaluate(x_test, y_test)
y_predict = model.predict(x_test)
y_predict=np.argmax(y_predict,axis=1)

# confusion matrix
cf_matrix = confusion_matrix(y_predict,y_test)

# classification report
print(classification_report(y_predict,y_test))
